# Game Log Sanitization

Strip out any personally-identifiable information from game log dump so data can be shared with the community.

In [1]:
import Pkg; 
Pkg.add("CSV");
Pkg.add("DataFrames");
Pkg.add("DataFramesMeta");

using CSV;
using DataFrames;
using DataFramesMeta;
import Dates;

    Updating registry at `/opt/julia/registries/General`
   Resolving package versions...
   Installed DataValueInterfaces ───────── v1.0.0
   Installed Tables ────────────────────── v1.4.2
   Installed SentinelArrays ────────────── v1.2.16
   Installed PooledArrays ──────────────── v1.2.1
   Installed DataAPI ───────────────────── v1.6.0
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed TableTraits ───────────────── v1.0.1
   Installed CSV ───────────────────────── v0.8.4
    Updating `/opt/julia/environments/v1.6/Project.toml`
  [336ed68f] + CSV v0.8.4
    Updating `/opt/julia/environments/v1.6/Manifest.toml`
  [336ed68f] + CSV v0.8.4
  [9a962f9c] + DataAPI v1.6.0
  [e2d170a0] + DataValueInterfaces v1.0.0
  [82899510] + IteratorInterfaceExtensions v1.0.0
  [2dfb63ee] + PooledArrays v1.2.1
  [91c51154] + SentinelArrays v1.2.16
  [3783bdb8] + TableTraits v1.0.1
  [bd369af6] + Tables v1.4.2
  [9fa8497b] + Future
Precompiling project...
  ✓ IteratorInterfaceExtensions
  ✓ Da

## Load the data

In [2]:
df = DataFrame(CSV.File("data.csv"));
size(df)

(84264, 9)

## Compute game length

In [3]:
fmt = Dates.DateFormat("yyyymmddTHHMMSSZ")
start_dt = Dates.DateTime.(df[:, :start], fmt)
end_dt = Dates.DateTime.(df[:, :end], fmt)
game_time = round.(end_dt - start_dt, Dates.Minute)
mins = map(x -> x.value, game_time)
insertcols!(df, 3, :time => mins)
size(df)

(84264, 10)

## Remove game time of day

In [4]:
transform!(df, :start => ByRow(x -> split(x, "T")[1]) => :date)
size(df)

(84264, 11)

## Write new CSV file

In [5]:
for_output = df[:, [:date, :time, :turn, :room, :format, :winner, :reason, :corp, :runner]]
fmt = Dates.DateFormat("yyyymmdd")
filename = string(Dates.format(Dates.now(), fmt), "_games.csv")
CSV.write(filename, for_output)

"20210507_games.csv"